# Generowanie syntetycznych danych o lekach

Ten notebook generuje syntetyczne dane o wymyślonych lekarstwach, które mogą być wykorzystane jako baza wiedzy w systemie RAG.
Dane zostaną utworzone przy użyciu GPT-4.1 i będą zawierały informacje takie jak:
- Nazwa leku (wymyślona)
- Wskazania do stosowania
- Dawkowanie dla dorosłych, dzieci i kobiet w ciąży
- Skutki uboczne
- Cena

Format danych będzie przypominał standardowe ulotki leków.

## Importowanie wymaganych bibliotek

In [ ]:
import random
from pathlib import Path
from openai import OpenAI
from pydantic import BaseModel, Field

# Sprawdzenie czy istnieje folder docelowy, jeśli nie - utworzenie go
output_dir = Path("008-02. RAG-data/drug-data")
output_dir.mkdir(exist_ok=True, parents=True)

## Konfiguracja klienta OpenAI

Upewnij się, że masz ustawioną zmienną środowiskową `OPENAI_API_KEY` z kluczem API OpenAI.

In [ ]:
client = OpenAI()

In [ ]:
drug_categories = [
    # Leki działające na układ sercowo-naczyniowy
    "Beta-blokery",
    "Inhibitory ACE",
    "Blokery kanału wapniowego",
    "Diuretyki",
    "Statyny",
    "Fibraty",
    "Antykoagulanty",
    "Leki przeciwpłytkowe",
    "Glikozydy nasercowe",
    "Leki przeciwarytmiczne",
    "Leki przeciwnadciśnieniowe",
    
    # Leki działające na układ nerwowy
    "Leki przeciwdepresyjne",
    "Leki przeciwpsychotyczne",
    "Leki przeciwpadaczkowe",
    "Leki przeciwparkinsonowskie",
    "Leki przeciwlękowe",
    "Leki nasenne i uspokajające",
    "Opioidy",
    "Leki przeciwbólowe nieopioidowe",
    "Leki przeciwmigrenowe",
    
    # Leki działające na układ oddechowy
    "Leki przeciwastmatyczne",
    "Leki rozszerzające oskrzela",
    "Leki przeciwkaszlowe",
    "Leki wykrztuśne",
    "Leki stosowane w POChP",
    "Leki przeciwalergiczne",
    
    # Leki działające na przewód pokarmowy
    "Inhibitory pompy protonowej",
    "Leki zobojętniające kwas żołądkowy",
    "Leki przeciwwymiotne",
    "Leki przeciwbiegunkowe",
    "Leki przeczyszczające",
    "Leki spazmolityczne",
    "Leki prokinetyczne",
    "Enzymy trzustkowe",
    
    # Leki przeciwinfekcyjne
    "Antybiotyki beta-laktamowe",
    "Makrolidy",
    "Tetracykliny",
    "Fluorochinolony",
    "Aminoglikozydy",
    "Leki przeciwgrzybicze",
    "Leki przeciwwirusowe",
    "Leki przeciwpasożytnicze",
    "Leki przeciwgruźlicze",
    
    # Leki immunomodulujące
    "Kortykosteroidy",
    "Leki immunosupresyjne",
    "Leki biologiczne",
    "Szczepionki",
    "Immunoglobuliny",
    
    # Leki hormonalne
    "Hormony tarczycy",
    "Leki przeciwcukrzycowe",
    "Insuliny",
    "Estrogeny",
    "Progestageny",
    "Androgeny",
    "Hormony wzrostu",
    "Hormonalne środki antykoncepcyjne",
    
    # Leki stosowane w chorobach metabolicznych
    "Leki stosowane w osteoporozie",
    "Leki przeciwdnawe",
    "Leki stosowane w chorobach rzadkich",
    
    # Leki działające na układ moczowy
    "Leki moczopędne",
    "Leki stosowane w przeroście prostaty",
    "Leki stosowane w nietrzymaniu moczu",
    
    # Leki dermatologiczne
    "Leki przeciwtrądzikowe",
    "Leki przeciwłuszczycowe",
    "Leki przeciwgrzybicze stosowane miejscowo",
    "Leki przeciwzapalne stosowane miejscowo",
    "Kortykosteroidy stosowane miejscowo",
    
    # Leki stosowane w okulistyce
    "Leki przeciwjaskrowe",
    "Leki zwężające naczynia krwionośne w oku",
    "Leki rozszerzające źrenice",
    "Leki przeciwzapalne stosowane w okulistyce",
    
    # Leki onkologiczne
    "Leki cytostatyczne",
    "Inhibitory kinaz",
    "Przeciwciała monoklonalne",
    "Leki hormonalne stosowane w onkologii",
    "Leki stosowane w terapii celowanej",
    "Immunoterapia nowotworów",
    
    # Leki stosowane w hematologii
    "Leki przeciwkrwotoczne",
    "Leki krwiotwórcze",
    "Czynniki krzepnięcia",
    
    # Suplementy i witaminy
    "Witaminy",
    "Minerały",
    "Suplementy wieloskładnikowe",
    "Preparaty ziołowe",
    
    # Pozostałe
    "Leki diagnostyczne",
    "Środki kontrastowe",
    "Leki stosowane w znieczuleniu",
    "Antidota",
    "Leki sieroce"
]

## Definicja struktury danych z Pydantic

Definiujemy modele Pydantic do kontroli struktury danych generowanych przez LLM.

In [ ]:
class Dosage(BaseModel):
    adults: str = Field(..., description="Dawkowanie dla dorosłych")
    children: str = Field(..., description="Dawkowanie dla dzieci z podziałem na grupy wiekowe")
    pregnancy: str = Field(..., description="Dawkowanie dla kobiet w ciąży i karmiących piersią")
    elderly: str = Field(..., description="Dawkowanie dla osób starszych")

class DrugInfo(BaseModel):
    name: str = Field(..., description="Nazwa produktu leczniczego")
    composition: str = Field(..., description="Skład jakościowy i ilościowy")
    indications: str = Field(..., description="Wskazania do stosowania")
    dosage: Dosage = Field(..., description="Dawkowanie i sposób podawania")
    contraindications: str = Field(..., description="Przeciwwskazania")
    warnings: str = Field(..., description="Specjalne ostrzeżenia i środki ostrożności")
    side_effects: str = Field(..., description="Działania niepożądane")
    storage: str = Field(..., description="Przechowywanie i okres ważności")
    manufacturer: str = Field(..., description="Podmiot odpowiedzialny")
    price: str = Field(..., description="Cena detaliczna")

## Definicja szablonów ulotki

In [ ]:
leaflet_templates = [
    # Szablon 1 - Standardowy
    """
NAZWA LEKU: {name}
=====================================

SKŁAD JAKOŚCIOWY I ILOŚCIOWY:
{composition}

WSKAZANIA DO STOSOWANIA:
{indications}

DAWKOWANIE I SPOSÓB PODAWANIA:
- Dorośli: {dosage_adults}
- Dzieci: {dosage_children}
- Kobiety w ciąży: {dosage_pregnancy}
- Osoby starsze: {dosage_elderly}

PRZECIWWSKAZANIA:
{contraindications}

SPECJALNE OSTRZEŻENIA I ŚRODKI OSTROŻNOŚCI:
{warnings}

DZIAŁANIA NIEPOŻĄDANE:
{side_effects}

PRZECHOWYWANIE I OKRES WAŻNOŚCI:
{storage}

PODMIOT ODPOWIEDZIALNY:
{manufacturer}

CENA DETALICZNA:
{price}
""",
    
    # Szablon 2 - Z podkreśleniami
    """
NAZWA LEKU: {name}
_____________________________________

SKŁAD:
{composition}
_____________________________________

WSKAZANIA:
{indications}
_____________________________________

DAWKOWANIE:
► Dorośli: {dosage_adults}
► Dzieci: {dosage_children}
► Ciąża i karmienie: {dosage_pregnancy}
► Seniorzy: {dosage_elderly}
_____________________________________

PRZECIWWSKAZANIA:
{contraindications}
_____________________________________

OSTRZEŻENIA:
{warnings}
_____________________________________

SKUTKI UBOCZNE:
{side_effects}
_____________________________________

PRZECHOWYWANIE:
{storage}
_____________________________________

PRODUCENT:
{manufacturer}

CENA: {price}
""",
    
    # Szablon 3 - Z gwiazdkami
    """
***** NAZWA LEKU: {name} *****

* SKŁAD *
{composition}

* WSKAZANIA *
{indications}

* DAWKOWANIE *
• Dorośli: {dosage_adults}
• Dzieci: {dosage_children}
• W ciąży: {dosage_pregnancy}
• Seniorzy: {dosage_elderly}

* PRZECIWWSKAZANIA *
{contraindications}

* OSTRZEŻENIA *
{warnings}

* DZIAŁANIA NIEPOŻĄDANE *
{side_effects}

* PRZECHOWYWANIE *
{storage}

* PRODUCENT *
{manufacturer}

* CENA DETALICZNA *
{price}
""",
    
    # Szablon 4 - Nowoczesny
    """
╔══════════════════════════════════╗
║        NAZWA LEKU: {name}      ║
╚══════════════════════════════════╝

▌SKŁAD
{composition}

▌WSKAZANIA
{indications}

▌DAWKOWANIE
  Dorośli: {dosage_adults}
  Dzieci: {dosage_children}
  Ciąża: {dosage_pregnancy}
  Seniorzy: {dosage_elderly}

▌PRZECIWWSKAZANIA
{contraindications}

▌OSTRZEŻENIA
{warnings}

▌SKUTKI UBOCZNE
{side_effects}

▌PRZECHOWYWANIE
{storage}

▌PRODUCENT
{manufacturer}

▌CENA: {price}
""",
    
    # Szablon 5 - Z punktami
    """
NAZWA LEKU: {name}
===============================

1. SKŁAD JAKOŚCIOWY I ILOŚCIOWY
{composition}

2. WSKAZANIA DO STOSOWANIA
{indications}

3. DAWKOWANIE I SPOSÓB PODAWANIA
   3.1. Dorośli: {dosage_adults}
   3.2. Dzieci: {dosage_children}
   3.3. Kobiety w ciąży: {dosage_pregnancy}
   3.4. Osoby starsze: {dosage_elderly}

4. PRZECIWWSKAZANIA
{contraindications}

5. SPECJALNE OSTRZEŻENIA I ŚRODKI OSTROŻNOŚCI
{warnings}

6. DZIAŁANIA NIEPOŻĄDANE
{side_effects}

7. PRZECHOWYWANIE I OKRES WAŻNOŚCI
{storage}

8. PODMIOT ODPOWIEDZIALNY
{manufacturer}

9. CENA DETALICZNA
{price}
""",
    
    # Szablon 6 - Z ramkami ASCII
    """
+-------------------------------------+
|        NAZWA LEKU: {name}           |
+-------------------------------------+

+-------------SKŁAD------------------+
{composition}
+-------------------------------------+

+------------WSKAZANIA--------------+
{indications}
+-------------------------------------+

+----------DAWKOWANIE---------------+
| Dorośli: {dosage_adults}
| Dzieci: {dosage_children}
| Ciąża: {dosage_pregnancy}
| Seniorzy: {dosage_elderly}
+-------------------------------------+

+--------PRZECIWWSKAZANIA-----------+
{contraindications}
+-------------------------------------+

+-----------OSTRZEŻENIA-------------+
{warnings}
+-------------------------------------+

+--------DZIAŁANIA NIEPOŻĄDANE------+
{side_effects}
+-------------------------------------+

+----------PRZECHOWYWANIE-----------+
{storage}
+-------------------------------------+

+----------PRODUCENT----------------+
{manufacturer}
+-------------------------------------+

+-----------CENA--------------------+
{price}
+-------------------------------------+
""",
    
    # Szablon 7 - Minimalny
    """
NAZWA LEKU: {name}

SKŁAD: {composition}

WSKAZANIA: {indications}

DAWKOWANIE:
Dorośli: {dosage_adults}
Dzieci: {dosage_children}
Ciąża: {dosage_pregnancy}
Seniorzy: {dosage_elderly}

PRZECIWWSKAZANIA: {contraindications}

OSTRZEŻENIA: {warnings}

SKUTKI UBOCZNE: {side_effects}

PRZECHOWYWANIE: {storage}

PRODUCENT: {manufacturer}

CENA: {price}
""",
    
    # Szablon 8 - Z symbolami strzałek
    """
➤ NAZWA LEKU: {name} ➤

➤ SKŁAD:
{composition}

➤ WSKAZANIA:
{indications}

➤ DAWKOWANIE:
  ↳ Dorośli: {dosage_adults}
  ↳ Dzieci: {dosage_children}
  ↳ Kobiety w ciąży: {dosage_pregnancy}
  ↳ Osoby starsze: {dosage_elderly}

➤ PRZECIWWSKAZANIA:
{contraindications}

➤ OSTRZEŻENIA:
{warnings}

➤ DZIAŁANIA NIEPOŻĄDANE:
{side_effects}

➤ PRZECHOWYWANIE:
{storage}

➤ PRODUCENT:
{manufacturer}

➤ CENA:
{price}
""",
    
    # Szablon 9 - Z cieniami
    """
█▓▒░ NAZWA LEKU: {name} ░▒▓█

░░ SKŁAD ░░
{composition}

░░ WSKAZANIA ░░
{indications}

░░ DAWKOWANIE ░░
▪ Dorośli: {dosage_adults}
▪ Dzieci: {dosage_children}
▪ Ciąża: {dosage_pregnancy}
▪ Seniorzy: {dosage_elderly}

░░ PRZECIWWSKAZANIA ░░
{contraindications}

░░ OSTRZEŻENIA ░░
{warnings}

░░ DZIAŁANIA NIEPOŻĄDANE ░░
{side_effects}

░░ PRZECHOWYWANIE ░░
{storage}

░░ PRODUCENT ░░
{manufacturer}

░░ CENA ░░
{price}
""",
    
    # Szablon 10 - Z symbolami medycznymi
    """
⚕️ NAZWA LEKU: {name} ⚕️

💊 SKŁAD:
{composition}

🏥 WSKAZANIA:
{indications}

⏱️ DAWKOWANIE:
👨‍👩‍👧‍👦 Dorośli: {dosage_adults}
👶 Dzieci: {dosage_children}
🤰 Ciąża: {dosage_pregnancy}
🧓 Seniorzy: {dosage_elderly}

⚠️ PRZECIWWSKAZANIA:
{contraindications}

⚠️ OSTRZEŻENIA:
{warnings}

😷 DZIAŁANIA NIEPOŻĄDANE:
{side_effects}

🧊 PRZECHOWYWANIE:
{storage}

🏢 PRODUCENT:
{manufacturer}

💰 CENA:
{price}
""",
]

## Definicja funkcji pomocniczych

In [ ]:
def extract_drug_name_from_filename(filename):
    """Wyodrębnia nazwę leku z nazwy pliku.
    
    Args:
        filename (str): Nazwa pliku w formacie 'NazwaLeku_X_mg_ml_postać.txt'
        
    Returns:
        str: Oryginalna nazwa leku
    """
    # Usuń rozszerzenie pliku
    name_without_ext = filename.rsplit('.', 1)[0]
    
    # Zastąp podkreślenia spacjami
    name = name_without_ext.replace('_', ' ')
    
    # Usuń ewentualne numery (dodawane przy duplikatach)
    if name[-2:].isdigit() and name[-3] == ' ':
        name = name[:-3]
        
    return name


def extract_brand_name(full_name):
    """Wyodrębnia samą nazwę handlową (brand) leku, bez dawkowania i postaci farmaceutycznej.
    
    Args:
        full_name (str): Pełna nazwa leku w formacie "NazwaHandlowa X mg/ml postać farmaceutyczna"
        
    Returns:
        str: Sama nazwa handlowa leku (brand)
    """
    import re
    
    # Typowe wzorce dawkowania
    dosage_patterns = [
        r'\d+(?:[,.]\d+)? mg',  # np. 10 mg, 2.5 mg
        r'\d+(?:[,.]\d+)? g',   # np. 1 g, 0.5 g
        r'\d+(?:[,.]\d+)? mcg', # np. 50 mcg
        r'\d+(?:[,.]\d+)? µg',  # np. 50 µg
        r'\d+(?:[,.]\d+)? mg/ml', # np. 5 mg/ml
        r'\d+(?:[,.]\d+)? mg/g', # np. 20 mg/g
        r'\d+ j\.? ?m\.?',  # np. 25000 j.m.
        r'\d+ IU',  # np. 100 IU
        r'\d+%',    # np. 2%
    ]
    
    # Połącz wszystkie wzorce za pomocą OR
    pattern = '|'.join(dosage_patterns)
    
    # Znajdź pierwsze wystąpienie dawkowania w nazwie
    match = re.search(pattern, full_name)
    
    if match:
        # Nazwa handlowa to tekst przed dawkowaniem
        brand_name = full_name[:match.start()].strip()
        return brand_name
    else:
        # Jeśli nie znaleziono wzorca dawkowania, zwróć pełną nazwę
        # (zazwyczaj nie powinno się zdarzyć, jeśli format jest przestrzegany)
        return full_name


def get_existing_drug_names(separate_brand_names=False):
    """Zbiera nazwy wszystkich istniejących leków z katalogu docelowego.
    
    Args:
        separate_brand_names (bool): Jeśli True, zwróci również listę samych nazw handlowych.
        
    Returns:
        tuple lub list: W zależności od separate_brand_names, zwraca listę pełnych nazw lub tuple (pełne_nazwy, nazwy_handlowe)
    """
    existing_names = []
    brand_names = set()  # Zbiór na unikalne nazwy handlowe
    
    # Sprawdź czy katalog docelowy istnieje
    if output_dir.exists():
        # Zbierz wszystkie pliki .txt z katalogu
        drug_files = list(output_dir.glob('*.txt'))
        
        for file_path in drug_files:
            # Wyodrębnij nazwę leku z nazwy pliku
            drug_name = extract_drug_name_from_filename(file_path.name)
            existing_names.append(drug_name)
            
            # Otwórz plik i odczytaj rzeczywistą nazwę leku z pierwszej linii
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read(500)  # Odczytaj pierwsze 500 znaków
                    
                    # Różne formaty mogą mieć różny sposób zapisu nazwy leku
                    if "NAZWA LEKU:" in content:
                        # Znajdź indeks początku nazwy leku
                        idx = content.find("NAZWA LEKU:") + len("NAZWA LEKU:")
                        # Wyodrębnij nazwę leku do końca linii lub określonej długości
                        line_end = content[idx:].find("\n")
                        if line_end > 0:
                            actual_name = content[idx:idx+line_end].strip()
                            if actual_name and actual_name not in existing_names:
                                existing_names.append(actual_name)
                                
                                # Dodaj samą nazwę handlową do zbioru, jeśli potrzebna
                                if separate_brand_names:
                                    brand_name = extract_brand_name(actual_name)
                                    brand_names.add(brand_name)
            except Exception as e:
                print(f"Błąd podczas odczytywania pliku {file_path.name}: {str(e)}")
    
    if separate_brand_names:
        return existing_names, list(brand_names)
    else:
        return existing_names

In [ ]:
def generate_and_save_drug_info(category=None, existing_drug_names=None, required_letters=None):
    """Generuje informacje o jednym wymyślonym leku z danej kategorii przy użyciu GPT-4.1 i zapisuje je do pliku.
    
    Args:
        category (str, optional): Kategoria leku do wygenerowania. Jeśli nie podana, zostanie wybrana losowo.
        existing_drug_names (list, optional): Lista nazw istniejących leków, aby uniknąć duplikatów.
        required_letters (list, optional): Lista liter, które muszą znaleźć się w nazwie leku.
        
    Returns:
        Path: Ścieżka do zapisanego pliku
    """
    
    # Jeśli kategoria nie jest podana, wybierz losową
    if category is None:
        category = random.choice(drug_categories)
    
    # Jeśli lista istniejących nazw leków nie jest podana, inicjuj pustą listę
    if existing_drug_names is None:
        existing_drug_names = []
        
    # Jeśli lista wymaganych liter nie jest podana, inicjuj pustą listę
    if required_letters is None:
        required_letters = []
    
    print(f"Generowanie i zapisywanie informacji o leku z kategorii: {category}...")
    
    system_prompt = """
    Jesteś ekspertem farmaceutycznym, który tworzy szczegółowe informacje o wymyślonych lekach.
    Dla każdego leku należy wygenerować kompletne informacje w formacie JSON zgodnie z podanym schematem.
    Informacje powinny być w języku polskim i zawierać następujące dane:
    
    1. name: Nazwa produktu leczniczego - wymyśl unikalną, wiarygodnie brzmiącą nazwę w formacie "NazwaHandlowa X mg/ml postać farmaceutyczna"
    2. composition: Skład jakościowy i ilościowy - wymyśl główne składniki aktywne i ich stężenia
    3. indications: Wskazania do stosowania - wymyśl szczegółowe informacje o schorzeniach, w których lek jest stosowany
    4. dosage: Obiekt zawierający informacje o dawkowaniu dla różnych grup:
       - adults: Dawkowanie dla dorosłych
       - children: Dawkowanie dla dzieci (z podziałem na grupy wiekowe)
       - pregnancy: Dawkowanie dla kobiet w ciąży i karmiących piersią
       - elderly: Dawkowanie dla osób starszych
    5. contraindications: Przeciwwskazania - wymień sytuacje, w których lek nie powinien być stosowany
    6. warnings: Specjalne ostrzeżenia i środki ostrożności
    7. side_effects: Działania niepożądane - wymień możliwe skutki uboczne z częstotliwością ich występowania
    8. storage: Przechowywanie i okres ważności
    9. manufacturer: Podmiot odpowiedzialny - wymyśl nazwę fikcyjnej firmy farmaceutycznej
    10. price: Cena detaliczna w złotych polskich (PLN)
    
    Odpowiedź MUSI być w formacie JSON zgodnym z podanym schematem.
    Utwórz realistyczne, ale wymyślone leki - nie kopiuj prawdziwych farmaceutyków.
    """
        
    try:
        # Przygotowanie listy istniejących leków do wykluczenia (dla lepszej czytelności w prompt)
        existing_drugs_str = "\n".join([f"- {name}" for name in existing_drug_names[:50]]) if existing_drug_names else "brak"
        
        # Przygotowanie informacji o wymaganych literach w nazwie
        required_letters_str = ", ".join(required_letters) if required_letters else "brak szczególnych wymagań"
        
        # Modyfikujemy prompt dla użytkownika, aby uwzględnić wybraną kategorię i istniejące leki
        user_prompt = f"""Wygeneruj szczegółowe informacje dla jednego wymyślonego leku w formacie JSON z kategorii: {category}.
        
        WAŻNE: Nazwa leku powinna być w formacie "NazwaHandlowa X mg/ml postać farmaceutyczna", 
        np. "Cardiofix 10 mg tabletki powlekane" lub "Neuroxol 5 mg/ml roztwór do wstrzykiwań". 
        
        WAŻNE: Nazwa handlowa leku MUSI zawierać następujące litery: {required_letters_str}.
        
        WAŻNE: NIE generuj leku o nazwie, która już istnieje w systemie. Poniżej lista istniejących nazw leków (unikaj ich):
        {existing_drugs_str}
        
        Zwróć dane WYŁĄCZNIE w formacie JSON"""
        
        response = client.beta.chat.completions.parse(
            model="gpt-4.1",  # lub najnowsza dostępna wersja
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0,
            max_tokens=5000,
            response_format=DrugInfo  # Wymuszenie formatu JSON
        )
        
        drug_info = response.choices[0].message.parsed
        
        # Wyodrębnij nazwę leku do nazwy pliku
        name = drug_info.name
        
        # Sprawdź, czy lek o takiej nazwie już istnieje
        if name in existing_drug_names:
            print(f"UWAGA: Wygenerowano duplikat nazwy leku: {name}. Ponawiam próbę...")
            # Rekurencyjnie generuj inny lek
            extended_list = existing_drug_names.append(name)
            return generate_and_save_drug_info(category, extended_list, required_letters)
        
        # Usunięcie znaków zakazanych w nazwach plików
        filename = ''.join(c if c.isalnum() or c in [' ', '-', '_'] else '_' for c in name)
        filename = filename.strip().replace(' ', '_')[:75]
        
        # Sprawdzenie czy plik o takiej nazwie już istnieje i ewentualnie dodanie sufiksu
        file_path = output_dir / f"{filename}.txt"
        counter = 1
        while file_path.exists():
            file_path = output_dir / f"{filename}_{counter}.txt"
            counter += 1
        
        # Wybór losowego szablonu ulotki
        selected_template = random.choice(leaflet_templates)
        
        # Formatowanie treści według wybranego szablonu
        formatted_content = selected_template.format(
            name=drug_info.name,
            composition=drug_info.composition,
            indications=drug_info.indications,
            dosage_adults=drug_info.dosage.adults,
            dosage_children=drug_info.dosage.children,
            dosage_pregnancy=drug_info.dosage.pregnancy,
            dosage_elderly=drug_info.dosage.elderly,
            contraindications=drug_info.contraindications,
            warnings=drug_info.warnings,
            side_effects=drug_info.side_effects,
            storage=drug_info.storage,
            manufacturer=drug_info.manufacturer,
            price=drug_info.price
        )
        
        # Zapisanie informacji do pliku w formacie ulotki
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(formatted_content)
        
        print(f"Zapisano informacje o leku '{name}' do pliku: {file_path}")
        return file_path, name
        
    except Exception as e:
        print(f"Błąd podczas przetwarzania danych leku: {str(e)}")
        return None, None

## Generowanie i zapisywanie danych o lekach

Poniższy kod wygeneruje i zapisze informacje o wymyślonych lekach. Możesz zmienić parametr `num_drugs`, aby wygenerować więcej lub mniej danych.

In [ ]:
# Liczba leków do wygenerowania
num_drugs = 10

In [ ]:
# Pobierz istniejące nazwy leków
existing_drug_names = get_existing_drug_names()
print(f"Znaleziono {len(existing_drug_names)} istniejących leków w katalogu docelowym.")

In [ ]:
# Generowanie i zapisywanie informacji o lekach
print(f"Rozpoczynam generowanie i zapisywanie informacji o {num_drugs} lekach...")

generated_names = []  # Lista nazw leków wygenerowanych w tej sesji
alphabet = 'abcdefghijklmnopqrstuvwxyz'

for i in range(num_drugs):
    # Losowe wybranie kategorii leku
    selected_category = random.choice(drug_categories)
    print(f"\nLek {i+1}/{num_drugs}:")
    
    # Losowy wybór 1-3 liter, które muszą znaleźć się w nazwie
    required_letters = random.sample(alphabet, 4)
    print(f"Wymagane litery w nazwie: {', '.join(required_letters)}")
    
    # Generowanie jednego leku z wybranej kategorii
    # Przekazanie zarówno istniejących nazw jak i nazw wygenerowanych w tej sesji
    all_drug_names = existing_drug_names + generated_names
    file_path, drug_name = generate_and_save_drug_info(selected_category, all_drug_names, required_letters)
    
    if file_path:
        generated_names.append(drug_name)

print(f"W bazie znajduje się teraz łącznie {len(existing_drug_names) + len(generated_names)} unikalnych leków.")